<a href="https://colab.research.google.com/github/GersteinJo/cat_mod/blob/main/modules_for_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modules

## metrics

In [1]:
!pip install git+https://github.com/GersteinJo/cat_mod.git

  Cloning https://github.com/GersteinJo/cat_mod.git to /tmp/pip-req-build-_sxukqor
  Running command git clone --filter=blob:none --quiet https://github.com/GersteinJo/cat_mod.git /tmp/pip-req-build-_sxukqor
  Resolved https://github.com/GersteinJo/cat_mod.git to commit 80faa4ce9135ef3e6ffff02fc2150503148f5d00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for cat_mod: filename=cat_mod-0.2-py3-none-any.whl size=6308 sha256=02b1d01b022ca11cdcbd6a36b7ad1687c81ca8176729337a3af0df0ec7541c0d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3jippbhu/wheels/d0/04/0a/ac2d69b1f4479d2a360a3c721c0958ecf9c2c81d1121682b81
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=30ffa4a92cf93a756ea2bd5c4ae8e4593b309f4dc68f312f81a0ae4f11aff597
  Stored in directory: /root/.cache/pip/wheels/19/db/95/5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built cat_mod minisom


In [2]:
from cat_mod import SEP
# from residual.loader import ImageLabelDataset

import numpy as np
import torch
from scipy.stats import pearsonr

#plotting
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import Viridis256
from bokeh.layouts import row

import plotly.express as px

def train_classifier(config, peck_obs_num = 200, n_iter = 10000, encoded_data=None, labels=None, logger = None):
    shuffle_mask = np.arange(len(labels))
    np.random.shuffle(shuffle_mask)

    sep = SEP.SEP(config['num_exemp'], output_space_shape=2, input_space_shape = config['input_space_shape'],
    lr = config['lr'], dr=config['dr'], omega = config['omega'], delta = config['delta'])
    results = np.zeros(peck_obs_num)
    peck = [1]
    counter = 0

    assert n_iter < len(labels), "Number of iterations should not surpass number of datapoints!"
    for i in range(n_iter):

        obs, label = encoded_data[shuffle_mask[i]].reshape(1,-1), labels[shuffle_mask[i]:shuffle_mask[i]+1]
        # print(label)
        peck = sep.predict(obs)

        if peck[0]==1:# and not (logger is None):
            # print("HEHEHEHEHEHHHHEHHEHEHEHEHEHEH")
            if logger:
                logger.log(
                    {"peck_edible": label[0]},
                    step = counter
                    )
            results[counter] = label.item()
            counter += 1
            if counter == peck_obs_num: print(f"Trial {i} finished!"); break
            sep.fit(obs, label)
        elif np.random.random() < 0.05:
            # print(f"label {label}")
            sep.fit(obs, label)

    return sep



def compare_embeddings(emb1, emb2):
    dissim1 = 1. - np.corrcoef(emb1)
    dissim2 = 1. - np.corrcoef(emb2)

    triu_indices = np.triu_indices_from(dissim1, k=1)
    flat1 = dissim1[triu_indices]
    flat2 = dissim2[triu_indices]

    # Compute second-order similarity (Pearson correlation)
    r, _ = pearsonr(flat1, flat2)
    return r

def embedding_plotter(embedding, data=None, hue=None, hover=None, tools = None, nv_cat = 5, height = 400, width = 400, display_result=True):
    '''
    Рисовалка эмбеддинга. 2D renderer: bokeh. 3D renderer: plotly.
    Обязательные инструменты:
        - pan (двигать график)
        - box zoom
        - reset (вылезти из зума в начальное положение)

        embedding: something 2D/3D, slicable ~ embedding[:, 0] - валидно
            Эмбеддинг
        data: pd.DataFrame
            Данные, по которым был построен эмбеддинг
        hue: string
            Колонка из data, по которой красим точки. Поддерживает интерактивную легенду: по клику на каждое
                значение hue можно скрыть весь цвет.
        hover: string or list of strings
            Колонк[а/и] из data, значения которых нужно выводить при наведении мышки на точку
        nv_cat: int
            number of unique values to consider column categorical
        tools: iterable or string in form "tool1,tool2,..." or ["tool1", "tool2", ...]
            tools for the interactive plot
        height, width: int
            parameters of the figure
        display_result: boolean
            if the results are displayed or just returned

    '''
    if tools is None:
        tools = 'lasso_select,box_select,pan,zoom_in,zoom_out,reset,hover'
    else:
        if hover and not("hover" in tools):
            tools = 'hover,'+",".join(tools)


    if embedding.shape[1] == 3:
        if hover:
            hover_data = {h:True for h in hover}
        else:
            hover_data = None
        df = pd.DataFrame(embedding, columns = ['x', 'y', 'z'])
        df = pd.concat((df, data), axis=1)
        fig = px.scatter_3d(
            data_frame = df,
            x='x',
            y='y',
            z='z',
            color=df[hue],
            hover_data = hover_data
        )

        fig.update_layout(
            modebar_add=tools.split(","),
        )

        fig.update_traces(marker_size=1, selector=dict(type='scatter3d'))

        if display_result: fig.show()

    if embedding.shape[1] == 2:
        output_notebook()
        df = pd.DataFrame(embedding, columns = ['x', 'y'])
        df = pd.concat((df, data), axis=1)
        tooltips = [
            ('x, y', '$x, $y'),
            ('index', '$index')
        ]
        if hover:
            for col in hover:
                tooltips.append((col, "@"+col))
        fig = figure(tools=tools, width=width, height=height, tooltips=tooltips)
        if df[hue].nunique() < nv_cat or df[hue].dtype == "category":
            df[hue] = df[hue].astype(str)
            source = ColumnDataSource(df)
            color_mapper = factor_cmap(
            field_name=hue,
            palette='Category10_3',
            factors=df[hue].unique()
            )
            fig.scatter(
            x='x', y='y',
            color=color_mapper,
            source=source,
            legend_group=hue)

            fig.legend.location = 'bottom_left'
            fig.legend.click_policy = 'mute'
        else:
            source = ColumnDataSource(df)
            color_mapper = linear_cmap(
                field_name=hue,
                palette=Viridis256,
                low=min(df[hue]),
                high=max(df[hue]))
            fig.scatter(
                x='x', y='y',
                color=color_mapper,
                source=source)
            color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0,0), title = hue)
            fig.add_layout(color_bar, 'right')


        if display_result: show(fig)

    if embedding.shape[1] > 3:
        print("wrong species, doooooodes")
    else: return fig

## pipelines

In [3]:
def encode_dataset(encoder_function: callable, loader, original_loader, device="cpu"):
    all_embeddings = []
    all_labels = []
    all_original_images = []

    with torch.no_grad():
        for (images_dino, targets), (images_orig, _) in zip(loader, original_loader):
            images_dino = images_dino.to(device, non_blocking=True)
            embeddings = encoder_function(images_dino).cpu()
            all_embeddings.append(embeddings)
            all_labels.append(targets)
            all_original_images.append(images_orig)

    embeddings = torch.cat(all_embeddings)  # [N, D]
    labels = torch.cat(all_labels)
    original_images = torch.cat(all_original_images)
    original_images = original_images.reshape(original_images.shape[0], -1)

    return (embeddings.numpy(),
            labels.numpy(),
            original_images.numpy())

import yaml
import wandb


import pandas as pd
import numpy as np
from sklearn.manifold import TSNE


class Runner:
    def __init__(self,
                 encoder_function,
                 loader,
                 original_loader, # in case encoder needs somewhat modified version of original image to proceed
                 config_path = None,
                ):
        if config_path:
            with open(config_path, 'r') as file:
                self.config = yaml.safe_load(file)
                print(self.config)
        else:
            self.config = None
        self.encoder_function = encoder_function
        self.loader = loader
        self.original_loader = original_loader

    def run(self, run_name = None, device = "cpu", n_iter = 1, embeddings=None, labels=None, original_images=None):
        if embeddings is None or labels is None or original_images is None:
            embeddings, labels, original_images = encode_dataset(self.encoder_function, self.loader, self.original_loader, device)
            print(embeddings.shape, labels.shape, original_images.shape)
        for _ in range(n_iter):
            permutation = np.arange(len(labels))
            np.random.shuffle(permutation)
            if run_name is None : run_name = f'encoder_{self.config["encoder"]}_kernel_{self.config["kernel"]}_num_exemplars_{self.config["num_exemp"]}'
            logger = wandb.init(project = 'final_runs', config = self.config, name = run_name,
                tags = [self.config['model_name'], self.config['encoder'], self.config['kernel'], f"num_exemp_{self.config['num_exemp']}"]) if self.config else None

            sep = train_classifier(self.config, encoded_data=embeddings[permutation], labels=labels[permutation], logger = logger)

            logger.log({"second_order_similarity" : compare_embeddings(embeddings[permutation[:10000]], original_images[permutation[:10000]])})

            logger.finish()

        # Assuming your embeddings are in a variable called 'embeddings'
        # embeddings = np.random.rand(10000, 384)  # Example - replace with your actual embeddings

        # Create t-SNE
        tsne = TSNE(n_components=2, random_state=1,
                    init='pca', n_iter=5000,
                    metric='euclidean')

        # Fit and transform your data
        tsne_results = tsne.fit_transform(embeddings[:1000])

        # Prepare the data DataFrame correctly
        data_df = pd.DataFrame({
            'label': np.array(labels[:1000])  # Assuming you have labels
            # Add any other columns you want for hover information
        })

        data_df['tsne_x'] = tsne_results[:,0]
        data_df['tsne_y'] = tsne_results[:,1]
        data_df['preds'] = sep.predict(embeddings[:1000])
        data_df['preds_vs_labels'] = (data_df['label']*10+data_df['preds']).map({
            11 : "edible_correct",
            10 : "edible_wrong",
            1  : "inedible_wrong",
            0  : "inedible_correct"
        })

        # Call the plotting function correctly
        embedding_plotter(
            embedding=tsne_results,  # This should be your 2D t-SNE results (1000x2 array)
            data=data_df,            # This contains your labels and other metadata
            hue='preds_vs_labels',             # Column name in data_df to use for coloring
        )

        return embeddings, labels, original_images, data_df





# encoder


In [4]:
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn as nn


class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.c0 = nn.Conv2d(3, 64, kernel_size=4, stride=1)
        self.c1 = nn.Conv2d(64, 128, kernel_size=4, stride=1)
        self.c2 = nn.Conv2d(128, 256, kernel_size=4, stride=1)
        self.c3 = nn.Conv2d(256, 512, kernel_size=4, stride=1)
        self.l1 = nn.Linear(512*20*20, 64)

        self.b1 = nn.BatchNorm2d(128)
        self.b2 = nn.BatchNorm2d(256)
        self.b3 = nn.BatchNorm2d(512)

    def forward(self, x):
        h = F.relu(self.c0(x))
        features = F.relu(self.b1(self.c1(h)))
        h = F.relu(self.b2(self.c2(features)))
        h = F.relu(self.b3(self.c3(h)))
        encoded = self.l1(h.view(x.shape[0], -1))
        return encoded, features


class GlobalDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.c0 = nn.Conv2d(128, 64, kernel_size=3)
        self.c1 = nn.Conv2d(64, 32, kernel_size=3)
        self.l0 = nn.Linear(32 * 22 * 22 + 64, 512)
        self.l1 = nn.Linear(512, 512)
        self.l2 = nn.Linear(512, 1)

    def forward(self, y, M):
        h = F.relu(self.c0(M))
        h = self.c1(h)
        h = h.view(y.shape[0], -1)
        h = torch.cat((y, h), dim=1)
        h = F.relu(self.l0(h))
        h = F.relu(self.l1(h))
        return self.l2(h)


class LocalDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.c0 = nn.Conv2d(192, 512, kernel_size=1)
        self.c1 = nn.Conv2d(512, 512, kernel_size=1)
        self.c2 = nn.Conv2d(512, 1, kernel_size=1)

    def forward(self, x):
        h = F.relu(self.c0(x))
        h = F.relu(self.c1(h))
        return self.c2(h)


class PriorDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.l0 = nn.Linear(64, 1000)
        self.l1 = nn.Linear(1000, 200)
        self.l2 = nn.Linear(200, 1)

    def forward(self, x):
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        return torch.sigmoid(self.l2(h))


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(64, 15)
        self.bn1 = nn.BatchNorm1d(15)
        self.l2 = nn.Linear(15, 10)
        self.bn2 = nn.BatchNorm1d(10)
        self.l3 = nn.Linear(10, 10)
        self.bn3 = nn.BatchNorm1d(10)

    def forward(self, x):
        encoded, _ = x[0], x[1]
        clazz = F.relu(self.bn1(self.l1(encoded)))
        clazz = F.relu(self.bn2(self.l2(clazz)))
        clazz = F.softmax(self.bn3(self.l3(clazz)), dim=1)
        return clazz


class DeepInfoAsLatent(nn.Module):
    def __init__(self, run, epoch):
        super().__init__()
        model_path = Path(r'/content') / Path(str(run)) / Path('encoder' + str(epoch) + '.wgt')
        self.encoder = Encoder()
        self.encoder.load_state_dict(torch.load(str(model_path)))
        self.classifier = Classifier()

    def forward(self, x):
        z, features = self.encoder(x)
        z = z.detach()
        return self.classifier((z, features))


import torch
from torchvision.datasets.cifar import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from pathlib import Path
import statistics as stats
import argparse


class DeepInfoMaxLoss(nn.Module):
    def __init__(self, alpha=0.5, beta=1.0, gamma=0.1):
        super().__init__()
        self.global_d = GlobalDiscriminator()
        self.local_d = LocalDiscriminator()
        self.prior_d = PriorDiscriminator()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

    def forward(self, y, M, M_prime):

        # see appendix 1A of https://arxiv.org/pdf/1808.06670.pdf

        y_exp = y.unsqueeze(-1).unsqueeze(-1)
        y_exp = y_exp.expand(-1, -1, 26, 26)

        y_M = torch.cat((M, y_exp), dim=1)
        y_M_prime = torch.cat((M_prime, y_exp), dim=1)

        Ej = -F.softplus(-self.local_d(y_M)).mean()
        Em = F.softplus(self.local_d(y_M_prime)).mean()
        LOCAL = (Em - Ej) * self.beta

        Ej = -F.softplus(-self.global_d(y, M)).mean()
        Em = F.softplus(self.global_d(y, M_prime)).mean()
        GLOBAL = (Em - Ej) * self.alpha

        prior = torch.rand_like(y)

        term_a = torch.log(self.prior_d(prior)).mean()
        term_b = torch.log(1.0 - self.prior_d(y)).mean()
        PRIOR = - (term_a + term_b) * self.gamma

        return LOCAL + GLOBAL + PRIOR

## dataset

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
import pandas as pd
from PIL import Image
import numpy as np

class ImageLabelDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]  # Assuming first column is image names
        img_path = f"{self.img_dir}/{img_name}.png"
        image = Image.open(img_path).convert('RGB')
        label = self.labels.iloc[idx, 1]  # Assuming second column is labels

        if self.transform:
            image = self.transform(image)

        return image, label

# Pipeline test

In [6]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q "/content/drive/MyDrive/Data/test_data.zip" -d /content/
!unzip -q "/content/drive/MyDrive/Data/pretrain_data.zip" -d /content/
!mkdir different_color
!unzip -q "/content/drive/MyDrive/Data/different_color.zip" -d /content/different_color/
!mkdir different_shape
!unzip -q "/content/drive/MyDrive/Data/different_shape.zip" -d /content/different_shape/

embeddings, labels, original_images = None, None, None

Mounted at /content/drive


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder().to(device)
root = Path(r'/content/drive/MyDrive/DIM_weights')
enc_file = root / Path('cifar_pretrained150.wgt')
encoder.load_state_dict(torch.load(str(enc_file), map_location = device))

# pd.read_csv('image_info.csv')['edible'].astype(int).to_csv("test_is_edible.csv")
# dataset = ImageLabelDataset(
#     csv_file="/content/test_is_edible.csv",
#     img_dir="/content/im",
#     transform=transforms.ToTensor()  # Add any other transforms
# )

# pd.read_csv('/content/different_color/different_color/image_info_edible.csv')['edible'].astype(int).to_csv("test_is_edible_DC.csv")
# dataset = ImageLabelDataset(
#     csv_file="/content/test_is_edible_DC.csv",
#     img_dir="/content/different_color/different_color",
#     transform=transforms.ToTensor()  # Add any other transforms
# )

pd.read_csv('/content/different_shape/different_shape/image_info_edible.csv')['edible'].astype(int).to_csv("test_is_edible_DS.csv")
dataset = ImageLabelDataset(
    csv_file="/content/test_is_edible_DS.csv",
    img_dir="/content/different_shape/different_shape",
    transform=transforms.ToTensor()  # Add any other transforms
)

loader = DataLoader(dataset, batch_size=512)
loader_original = DataLoader(dataset, batch_size=512)

encoder_func = lambda x: encoder(x)[0]
print(len(loader))
runner = Runner(
    encoder_func,
    loader,
    loader_original,
    "/content/DIM_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("different_shape", device, n_iter = 40, embeddings = embeddings, labels = labels, original_images = original_images)


40
{'model_name': 'SEP', 'encoder': 'DIM', 'input_space_shape': 64, 'delta': 1.5, 'lr': 0.005, 'dr': 0.995, 'omega': 0.5, 'kernel': 'exponential', 'num_exemp': 30, 'dataset': None}
(20000, 64) (20000,) (20000, 3072)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ms-hate-life (ms-hate-life-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

Trial 682 finished!


peck_edible,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.71865


Trial 659 finished!


peck_edible,███████████████▁████████████████████████
second_order_similarity,▁
peck_edible,0
second_order_similarity,0.71475


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [9]:
import pandas as pd

df = pd.DataFrame(np.hstack([embeddings, original_images]), columns = [f"emb_{i}" for i in range(64)]+[f"or_{i}" for i in range(32*32*3)])
df['label'] = labels
df.to_csv('/content/drive/MyDrive/Data/DIM_embedding_DC.csv')

tsne_results.to_csv('/content/drive/MyDrive/Data/DIM_tsne_embedding_DC.csv')

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

df = pd.read_csv('/content/drive/MyDrive/Data/DIM_embedding.csv')

In [20]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

runner = Runner(
    encoder_function = None,
    loader = None,
    original_loader = None,
    config_path = "/content/DIM_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("DIM_runs_05_15", embeddings = embeddings, labels = labels, original_images = original_images, n_iter = 50)




{'model_name': 'SEP', 'encoder': 'DIM', 'input_space_shape': 64, 'delta': 1.5, 'lr': 0.005, 'dr': 0.995, 'omega': 0.5, 'kernel': 'exponential', 'num_exemp': 30}


Trial 2190 finished!


peck_edible,██████▁████▁███████████████▁████▁███████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.82755


Trial 2032 finished!


peck_edible,███▁███████████████████████████████████▁
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.82768


...

Trial 2693 finished!


peck_edible,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.82828


Trial 2179 finished!


peck_edible,▁███▁███████▁█▁█████████████▁█████████▁█
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.82692


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [19]:
wandb.finish()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import Viridis256
from bokeh.layouts import row

import plotly.express as px

def embedding_plotter(embedding, data=None, hue=None, hover=None, tools = None, nv_cat = 5, height = 400, width = 400, display_result=True):
    '''
    Рисовалка эмбеддинга. 2D renderer: bokeh. 3D renderer: plotly.
    Обязательные инструменты:
        - pan (двигать график)
        - box zoom
        - reset (вылезти из зума в начальное положение)

        embedding: something 2D/3D, slicable ~ embedding[:, 0] - валидно
            Эмбеддинг
        data: pd.DataFrame
            Данные, по которым был построен эмбеддинг
        hue: string
            Колонка из data, по которой красим точки. Поддерживает интерактивную легенду: по клику на каждое
                значение hue можно скрыть весь цвет.
        hover: string or list of strings
            Колонк[а/и] из data, значения которых нужно выводить при наведении мышки на точку
        nv_cat: int
            number of unique values to consider column categorical
        tools: iterable or string in form "tool1,tool2,..." or ["tool1", "tool2", ...]
            tools for the interactive plot
        height, width: int
            parameters of the figure
        display_result: boolean
            if the results are displayed or just returned

    '''
    if tools is None:
        tools = 'lasso_select,box_select,pan,zoom_in,zoom_out,reset,hover'
    else:
        if hover and not("hover" in tools):
            tools = 'hover,'+",".join(tools)


    if embedding.shape[1] == 3:
        if hover:
            hover_data = {h:True for h in hover}
        else:
            hover_data = None
        df = pd.DataFrame(embedding, columns = ['x', 'y', 'z'])
        df = pd.concat((df, data), axis=1)
        fig = px.scatter_3d(
            data_frame = df,
            x='x',
            y='y',
            z='z',
            color=df[hue],
            hover_data = hover_data
        )

        fig.update_layout(
            modebar_add=tools.split(","),
        )

        fig.update_traces(marker_size=1, selector=dict(type='scatter3d'))

        if display_result: fig.show()

    if embedding.shape[1] == 2:
        output_notebook()
        df = pd.DataFrame(embedding, columns = ['x', 'y'])
        df = pd.concat((df, data), axis=1)
        tooltips = [
            ('x, y', '$x, $y'),
            ('index', '$index')
        ]
        if hover:
            for col in hover:
                tooltips.append((col, "@"+col))
        fig = figure(tools=tools, width=width, height=height, tooltips=tooltips)
        if df[hue].nunique() < nv_cat or df[hue].dtype == "category":
            df[hue] = df[hue].astype(str)
            source = ColumnDataSource(df)
            color_mapper = factor_cmap(
            field_name=hue,
            palette='Category10_3',
            factors=df[hue].unique()
            )
            fig.scatter(
            x='x', y='y',
            color=color_mapper,
            source=source,
            legend_group=hue)

            fig.legend.location = 'bottom_left'
            fig.legend.click_policy = 'mute'
        else:
            source = ColumnDataSource(df)
            color_mapper = linear_cmap(
                field_name=hue,
                palette=Viridis256,
                low=min(df[hue]),
                high=max(df[hue]))
            fig.scatter(
                x='x', y='y',
                color=color_mapper,
                source=source)
            color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0,0), title = hue)
            fig.add_layout(color_bar, 'right')


        if display_result: show(fig)

    if embedding.shape[1] > 3:
        print("wrong species, doooooodes")
    else: return fig

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE

# Assuming your embeddings are in a variable called 'embeddings'
# embeddings = np.random.rand(10000, 384)  # Example - replace with your actual embeddings

# Create t-SNE
tsne = TSNE(n_components=2, random_state=1,
            init='pca', n_iter=5000,
            metric='euclidean')

# Fit and transform your data
tsne_results = tsne.fit_transform(embeddings[:1000])

# Prepare the data DataFrame correctly
data_df = pd.DataFrame({
    'label': np.array(labels[:1000])  # Assuming you have labels
    # Add any other columns you want for hover information
})

# Call the plotting function correctly
embedding_plotter(
    embedding=tsne_results,  # This should be your 2D t-SNE results (1000x2 array)
    data=data_df,            # This contains your labels and other metadata
    hue='label',             # Column name in data_df to use for coloring
)

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


figure(id='p1001', ...)

In [29]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
print(embeddings.shape, labels.shape)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set
pred = tree.predict(embeddings[2000:3000])

# Calculate precision
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 64) (50000,)
Precision: 0.9375


In [30]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.linear_model import LogisticRegression  # Changed import
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

print(embeddings.shape, labels.shape)

# Changed to LogisticRegression
tree = LogisticRegression(random_state=42, max_iter=1000)  # Added max_iter for convergence
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set (same as before)
pred = tree.predict(embeddings[2000:3000])

# Calculate precision (same as before)
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 64) (50000,)
Precision: 0.9915


# VAE

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


from pathlib import Path

class ConvVAE(nn.Module):
    def __init__(self, latent_dim=128):
        super(ConvVAE, self).__init__()

        # Encoder (for 32x32 CIFAR images)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 32x32 -> 16x16
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),

            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 16x16 -> 8x8
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 8x8 -> 4x4
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )

        # Latent space
        self.fc_mu = nn.Linear(128 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(128 * 4 * 4, latent_dim)

        # Decoder
        self.decoder_fc = nn.Linear(latent_dim, 128 * 4 * 4)

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # 16x16 -> 32x32
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),

            nn.Conv2d(32, 3, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        z = self.decoder_fc(z)
        z = z.view(z.size(0), 128, 4, 4)
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar

def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss (MSE works better for CIFAR than BCE)
    MSE = nn.functional.mse_loss(recon_x, x, reduction='sum')

    # KL divergence
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + KL

def train(model, optimizer, train_loader, num_epochs, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    train_loss_rec = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_idx, data in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()

            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch+1}, Batch: {batch_idx}, Loss: {loss.item()/len(data):.4f}')

        avg_loss = train_loss / len(train_loader.dataset)
        train_loss_rec.append(avg_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

    plt.plot(np.arange(num_epochs), train_loss_rec)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.show()


# Hyperparameters
latent_dim = 128
num_epochs = 10
batch_size = 128
learning_rate = 1e-5

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = ConvVAE(latent_dim=latent_dim).to(device)
root = Path(r'/content/drive/MyDrive/VAE_weights')
enc_file = root / Path('cifar_pretrained119.wgt')
encoder.load_state_dict(torch.load(str(enc_file), map_location = device))

pd.read_csv('image_info.csv')['edible'].astype(int).to_csv("test_is_edible.csv")
dataset = ImageLabelDataset(
    csv_file="/content/test_is_edible.csv",
    img_dir="/content/im",
    transform=transforms.ToTensor()  # Add any other transforms
)
loader = DataLoader(dataset, batch_size=512)
loader_original = DataLoader(dataset, batch_size=512)

encoder_func = lambda x: encoder.encode(x)[0]
print(len(loader))
runner = Runner(
    encoder_func,
    loader,
    loader_original,
    "/content/exemple_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("test_rerunning", device, n_iter = 30)

import pandas as pd

df = pd.DataFrame(np.hstack([embeddings, original_images]), columns = [f"emb_{i}" for i in range(128)]+[f"or_{i}" for i in range(32*32*3)])
df['label'] = labels
df.to_csv('/content/drive/MyDrive/Data/VAE_embedding.csv')

tsne_results.to_csv('/content/drive/MyDrive/Data/VAE_tsne_embedding.csv')

<ipython-input-12-33919c872f74>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(str(enc_file), map_location = device))


98
{'model_name': 'SEP', 'encoder': 'VAE', 'input_space_shape': 128, 'delta': 15.0, 'lr': 0.1, 'dr': 0.9, 'omega': 0.3, 'kernel': 'exponential', 'num_exemp': 30}


KeyboardInterrupt: 

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

df = pd.read_csv('/content/drive/MyDrive/Data/VAE_embedding.csv')

In [18]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

runner = Runner(
    encoder_function = None,
    loader = None,
    original_loader = None,
    config_path = "/content/exemple_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("VAE_runs_15", embeddings = embeddings, labels = labels, original_images = original_images, n_iter = 45)


{'model_name': 'SEP', 'encoder': 'VAE', 'input_space_shape': 128, 'delta': 15.0, 'lr': 0.1, 'dr': 0.9, 'omega': 0.7, 'kernel': 'exponential', 'num_exemp': 30}


Trial 1097 finished!


peck_edible,█████▁█████████████████████▁████████████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.65092


Trial 966 finished!


peck_edible,██████▁▁███████▁▁████████████████▁██████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.64446


...

Trial 988 finished!


peck_edible,█████████▁███████▁▁█████████████████████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.63727


Trial 1164 finished!


peck_edible,███▁███████████████▁█████▁██████▁███████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.65001


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [32]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
print(embeddings.shape, labels.shape)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set
pred = tree.predict(embeddings[2000:3000])

# Calculate precision
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 128) (50000,)
Precision: 0.9013


In [33]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.linear_model import LogisticRegression  # Changed import
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

print(embeddings.shape, labels.shape)

# Changed to LogisticRegression
tree = LogisticRegression(random_state=42, max_iter=1000)  # Added max_iter for convergence
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set (same as before)
pred = tree.predict(embeddings[2000:3000])

# Calculate precision (same as before)
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 128) (50000,)
Precision: 0.8834


In [16]:
wandb.finish()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.palettes import Viridis256
from bokeh.layouts import row

import plotly.express as px

def embedding_plotter(embedding, data=None, hue=None, hover=None, tools = None, nv_cat = 5, height = 400, width = 400, display_result=True):
    '''
    Рисовалка эмбеддинга. 2D renderer: bokeh. 3D renderer: plotly.
    Обязательные инструменты:
        - pan (двигать график)
        - box zoom
        - reset (вылезти из зума в начальное положение)

        embedding: something 2D/3D, slicable ~ embedding[:, 0] - валидно
            Эмбеддинг
        data: pd.DataFrame
            Данные, по которым был построен эмбеддинг
        hue: string
            Колонка из data, по которой красим точки. Поддерживает интерактивную легенду: по клику на каждое
                значение hue можно скрыть весь цвет.
        hover: string or list of strings
            Колонк[а/и] из data, значения которых нужно выводить при наведении мышки на точку
        nv_cat: int
            number of unique values to consider column categorical
        tools: iterable or string in form "tool1,tool2,..." or ["tool1", "tool2", ...]
            tools for the interactive plot
        height, width: int
            parameters of the figure
        display_result: boolean
            if the results are displayed or just returned

    '''
    if tools is None:
        tools = 'lasso_select,box_select,pan,zoom_in,zoom_out,reset,hover'
    else:
        if hover and not("hover" in tools):
            tools = 'hover,'+",".join(tools)


    if embedding.shape[1] == 3:
        if hover:
            hover_data = {h:True for h in hover}
        else:
            hover_data = None
        df = pd.DataFrame(embedding, columns = ['x', 'y', 'z'])
        df = pd.concat((df, data), axis=1)
        fig = px.scatter_3d(
            data_frame = df,
            x='x',
            y='y',
            z='z',
            color=df[hue],
            hover_data = hover_data
        )

        fig.update_layout(
            modebar_add=tools.split(","),
        )

        fig.update_traces(marker_size=1, selector=dict(type='scatter3d'))

        if display_result: fig.show()

    if embedding.shape[1] == 2:
        output_notebook()
        df = pd.DataFrame(embedding, columns = ['x', 'y'])
        df = pd.concat((df, data), axis=1)
        tooltips = [
            ('x, y', '$x, $y'),
            ('index', '$index')
        ]
        if hover:
            for col in hover:
                tooltips.append((col, "@"+col))
        fig = figure(tools=tools, width=width, height=height, tooltips=tooltips)
        if df[hue].nunique() < nv_cat or df[hue].dtype == "category":
            df[hue] = df[hue].astype(str)
            source = ColumnDataSource(df)
            color_mapper = factor_cmap(
            field_name=hue,
            palette='Category10_3',
            factors=df[hue].unique()
            )
            fig.scatter(
            x='x', y='y',
            color=color_mapper,
            source=source,
            legend_group=hue)

            fig.legend.location = 'bottom_left'
            fig.legend.click_policy = 'mute'
        else:
            source = ColumnDataSource(df)
            color_mapper = linear_cmap(
                field_name=hue,
                palette=Viridis256,
                low=min(df[hue]),
                high=max(df[hue]))
            fig.scatter(
                x='x', y='y',
                color=color_mapper,
                source=source)
            color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0,0), title = hue)
            fig.add_layout(color_bar, 'right')


        if display_result: show(fig)

    if embedding.shape[1] > 3:
        print("wrong species, doooooodes")
    else: return fig

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE

# Assuming your embeddings are in a variable called 'embeddings'
# embeddings = np.random.rand(10000, 384)  # Example - replace with your actual embeddings

# Create t-SNE
tsne = TSNE(n_components=2, random_state=1,
            init='pca', n_iter=5000,
            metric='euclidean')

# Fit and transform your data
tsne_results = tsne.fit_transform(embeddings[:1000])

# Prepare the data DataFrame correctly
data_df = pd.DataFrame({
    'label': np.array(labels[:1000])  # Assuming you have labels
    # Add any other columns you want for hover information
})

# Call the plotting function correctly
embedding_plotter(
    embedding=tsne_results,  # This should be your 2D t-SNE results (1000x2 array)
    data=data_df,            # This contains your labels and other metadata
    hue='label',             # Column name in data_df to use for coloring
)

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


figure(id='p1058', ...)

# CNN untrained

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
class DeepCNNEncoder(nn.Module):
    def __init__(self):
        super(DeepCNNEncoder, self).__init__()

        self.encoder = nn.Sequential(
            # Block 1: 3x32x32 -> 32x32x32
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            # Block 2: 32x32x32 -> 32x32x32
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            # Downsample 1: 32x32x32 -> 64x16x16
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # Block 3: 64x16x16 -> 64x16x16
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # Block 4: 64x16x16 -> 64x16x16
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # Downsample 2: 64x16x16 -> 128x8x8
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # Block 5: 128x8x8 -> 128x8x8
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # Block 6: 128x8x8 -> 128x8x8
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # Final projection: 128x8x8 -> 1x8x8
            nn.Conv2d(128, 1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x).view(x.size(0), -1)

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = DeepCNNEncoder().to(device)

pd.read_csv('image_info.csv')['edible'].astype(int).to_csv("test_is_edible.csv")
dataset = ImageLabelDataset(
    csv_file="/content/test_is_edible.csv",
    img_dir="/content/im",
    transform=transforms.ToTensor()  # Add any other transforms
)
loader = DataLoader(dataset, batch_size=512)
loader_original = DataLoader(dataset, batch_size=512)

encoder_func = lambda x: encoder(x)
print(len(loader))
runner = Runner(
    encoder_func,
    loader,
    loader_original,
    "/content/CNN_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("CNN_runs", device, n_iter = 1)

98
{'model_name': 'SEP', 'encoder': 'DeepCNNEncoder', 'input_space_shape': 64, 'delta': 0.1, 'lr': 0.001, 'dr': 0.999, 'omega': 0.3, 'kernel': 'exponential', 'num_exemp': 20}
(50000, 64) (50000,) (50000, 3072)


Trial 290 finished!


peck_edible,▁███▁▁█▁█▁▁███████▁█▁▁█▁▁▁██▁█▁▁█▁▁███▁█
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.23603


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [ ]:
print(embeddings.shape, labels.shape, original_images.shape)

(50000, 64) (50000,) (50000, 3072)


In [37]:
import pandas as pd

df = pd.DataFrame(np.hstack([embeddings, original_images]), columns = [f"emb_{i}" for i in range(64)]+[f"or_{i}" for i in range(32*32*3)])
df['label'] = labels
df.to_csv('/content/drive/MyDrive/Data/CNN_embedding.csv')

tsne_results.to_csv('/content/drive/MyDrive/Data/tsne_CNN_embedding.csv')

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import sys

import pandas as pd

from torchvision import transforms

df = pd.read_csv('/content/drive/MyDrive/Data/CNN_embedding.csv')


In [8]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

runner = Runner(
    encoder_function = None,
    loader = None,
    original_loader = None,
    config_path = "/content/CNN_config.yaml"
)

embeddings, labels, original_images, tsne_results = runner.run("CNN_runs_10_", embeddings = embeddings, labels = labels, original_images = original_images, n_iter = 40)

{'model_name': 'SEP', 'encoder': 'DeepCNNEncoder', 'input_space_shape': 64, 'delta': 10.0, 'lr': 0.001, 'dr': 0.999, 'omega': 0.3, 'kernel': 'exponential', 'num_exemp': 30}


Trial 561 finished!


peck_edible,████████████▁██████▁███████████████████▁
second_order_similarity,▁
peck_edible,0
second_order_similarity,0.24782


Trial 719 finished!


peck_edible,███▁█▁████▁███▁█▁▁████████████▁█████████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.25188


...

Trial 607 finished!


peck_edible,█▁▁███▁█▁████████▁▁█████▁███████████████
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.24616


Trial 599 finished!


peck_edible,█▁█████▁██████▁▁████████▁█▁████████▁█▁██
second_order_similarity,▁
peck_edible,1
second_order_similarity,0.25031


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [44]:
wandb.finish()

In [38]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
print(embeddings.shape, labels.shape)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set
pred = tree.predict(embeddings[2000:3000])

# Calculate precision
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 64) (50000,)
Precision: 0.7883


In [39]:
embeddings, labels, original_images = df[[c for c in df.columns if 'emb' in c]].to_numpy(), df['label'].to_numpy(), df[[c for c in df.columns if 'or' in c]].to_numpy()

from sklearn.linear_model import LogisticRegression  # Changed import
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

print(embeddings.shape, labels.shape)

# Changed to LogisticRegression
tree = LogisticRegression(random_state=42, max_iter=1000)  # Added max_iter for convergence
tree.fit(embeddings[:2000], labels[:2000])

# Make predictions on the test set (same as before)
pred = tree.predict(embeddings[2000:3000])

# Calculate precision (same as before)
precision = precision_score(labels[2000:3000], pred)

print(f"Precision: {precision:.4f}")

(50000, 64) (50000,)
Precision: 0.8827
